# Pulling Data from eia gov via REST API


# CO2 EMISSIONS

In [1]:
# Ensure your API key is stored in this module
import config  
import requests
import json
import pandas as pd
import csv
import os

# access api key
api_key = config.API_KEY


# function that tests if the api end point is working
def test_API(url):
    response = requests.get(url)
    if response.status_code == 200:
        print("\n\nAPI key is working, Response:", response.status_code)
    else:
        print("\n\nError with API request, Response:", response.status_code)


# write to CSV file func
def write_data_to_csv(data, directory, filename='output.csv'):
    
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)
    
    # full path to the file
    file_path = os.path.join(directory, filename)
    
    # Write the data to file
    with open(file_path, mode='w', newline='') as csv_file:
        
        # Get the header from the keys 
        fieldnames = data[0].keys()  
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write the header
        writer.writeheader()  
        for item in data:
            writer.writerow(item)  # Write data rows

    print(f"Data written to {file_path}")




def emission_Rank_Calc(df, value_column='value', group_by_column=['state-name', 'stateId'], top_n=10):
    """
    Calculate the CO2 emission rank for states based on the provided DataFrame.

    Parameters:
    - df: pd.DataFrame containing emissions data.
    - value_column: str, the name of the column containing the emissions values.
    - group_by_column: str, the name of the column to group by (e.g., state names).
    - top_n: int, the number of top states to return.

    Returns:
    - pd.DataFrame containing the top N states ranked by CO2 emissions.
    """
    # Convert value column to numeric type
    df[value_column] = pd.to_numeric(df[value_column], errors='coerce')

    # Aggregate total CO2 emissions by state
    state_emissions = df.groupby(group_by_column)[value_column].sum().reset_index()

    # Sort emissions dsc
    state_emissions = state_emissions.sort_values(by=value_column, ascending=False)

    # Rank states
    state_emissions['Rank'] = state_emissions[value_column].rank(ascending=False, method='min').astype(int)

    # Sort by rank
    state_emissions = state_emissions.sort_values(by='Rank')

    # top N states by CO2 emissions
    return state_emissions.head(top_n)

## Pull data without filter

In [2]:
# Define the base URL and parameters
url = "https://api.eia.gov/v2/co2-emissions/co2-emissions-aggregates/data/"
api_key = config.API_KEY

# base url = url + api_key
base_url_ = f"{url}?api_key={api_key}"


# test the API / access key if working
print("test api ...")
test_API(base_url_)


# Parse the JSON response to check the response
full_response = requests.get(base_url_).json()


# full data response keys
print("\nkeys on the response: ", list(full_response.keys()))

# response elements
print("\napiVersion:", full_response['apiVersion'])

print("\nExcelAddInVersion:", full_response['ExcelAddInVersion'])

test api ...


API key is working, Response: 200

keys on the response:  ['response', 'request', 'apiVersion', 'ExcelAddInVersion']

apiVersion: 2.1.7

ExcelAddInVersion: 2.1.0


In [3]:
full_response_data = full_response['response']['data']



# full response total count
print("\nfull response total count: ", full_response['response']['total'])


# full response warning
print("\nwarning description: ",full_response['response']['warnings'][0]['description'])

# number of records on full reponse data 
print("\nfull response record count: ", len(full_response_data))


full response total count:  64896

warning description:  The API can only return 5000 rows in JSON format.  Please consider constraining your request with facet, start, or end, or using offset to paginate results.

full response record count:  5000


#### There are 64896 records, only 5000 returned. API returns max of 5000 records. 

# Pull filtered data
- year = `2019`
- states = all `50` states ( excluded USA and Wahington DC which are not states)
- sectorID = `'TT'`
- fuelID = `all fules` (not filterd)

The API dashborad generates a unique URL endpoint for the above filter criteria. 

In [4]:
url = "https://api.eia.gov/v2/co2-emissions/co2-emissions-aggregates/data/"
api_key = config.API_KEY


# base url = url + api_key
url_0 = f"{url}?api_key={api_key}"

params = {
    "frequency": "annual",
    "data[0]": "value",
    "facets[sectorId][]": "TT",
    "facets[stateId][]": [
        "AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", 
        "HI", "IA", "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", 
        "ME", "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE", "NH", 
        "NJ", "NM", "NV", "NY", "OH", "OK", "OR", "PA", "RI", "SC", 
        "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI", "WV", "WY"
    ],
    "start": 2019,
    "end": 2019,
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
}

# Check the response
test_API(url_0)

# filtered response
filtered_response_co2_2019 = requests.get(url_0, params=params).json()


# total filtered response
print("\nrecord count for filtered response: ", filtered_response_co2_2019['response']['total'])


# extract the actual data
data_2019 = filtered_response_co2_2019['response']['data']


# view sample data
print("\n\nview the first 2 records:")
data_2019[:2]



API key is working, Response: 200

record count for filtered response:  200


view the first 2 records:


[{'period': '2019',
  'sectorId': 'TT',
  'sector-name': 'Total carbon dioxide emissions from all sectors',
  'fuelId': 'PE',
  'fuel-name': 'Petroleum',
  'stateId': 'AK',
  'state-name': 'Alaska',
  'value': '15.068522',
  'value-units': 'million metric tons of CO2'},
 {'period': '2019',
  'sectorId': 'TT',
  'sector-name': 'Total carbon dioxide emissions from all sectors',
  'fuelId': 'CO',
  'fuel-name': 'Coal',
  'stateId': 'AK',
  'state-name': 'Alaska',
  'value': '1.685664',
  'value-units': 'million metric tons of CO2'}]

## Write filtered data 2019 of 50 states to csv file

In [5]:
write_data_to_csv(data_2019, 
                  directory='Data/CO2/', 
                  filename='co2_emissions_2019.csv')

Data written to Data/CO2/co2_emissions_2019.csv


### Import to pandas df

In [6]:
import pandas as pd
import os



# co2
df = pd.read_csv('Data/CO2/co2_emissions_2019.csv')
df

,period,sectorId,sector-name,fuelId,fuel-name,stateId,state-name,value,value-units
0,2019,TT,Total carbon dioxide emissions from all sectors,PE,Petroleum,AK,Alaska,15.068522,million metric tons of CO2
1,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,AK,Alaska,1.685664,million metric tons of CO2
2,2019,TT,Total carbon dioxide emissions from all sectors,NG,Natural Gas,AK,Alaska,17.522644,million metric tons of CO2
3,2019,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,AK,Alaska,34.276830,million metric tons of CO2
4,2019,TT,Total carbon dioxide emissions from all sectors,PE,Petroleum,AL,Alabama,36.945132,million metric tons of CO2
...,...,...,...,...,...,...,...,...,...
195,2019,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,WV,West Virginia,85.450237,million metric tons of CO2
196,2019,TT,Total carbon dioxide emissions from all sectors,PE,Petroleum,WY,Wyoming,11.040071,million metric tons of CO2
197,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,WY,Wyoming,39.227048,million metric tons of CO2
198,2019,TT,Total carbon dioxide emissions from all sectors,NG,Natural Gas,WY,Wyoming,8.830283,million metric tons of CO2


## Emission Rank by state calculation 

In [7]:
#def emission_Rank_Calc(df, value_column='value', group_by_column='state-name', top_n=10):

top_10_states_co2 = emission_Rank_Calc(df, top_n=10)

top_10_states_co2

,state-name,stateId,value,Rank
42,Texas,TX,1366.830499,1
4,California,CA,716.532710,2
8,Florida,FL,467.235577,3
37,Pennsylvania,PA,437.515984,4
12,Illinois,IL,406.950183,5
34,Ohio,OH,393.398570,6
17,Louisiana,LA,389.847568,7
13,Indiana,IN,352.363650,8
31,New York,NY,338.052048,9
21,Michigan,MI,318.511772,10


## North Dakota CO2 emission

In [8]:

tall_states_co2 = emission_Rank_Calc(df, top_n=50)

north_dakota_emissions = tall_states_co2[tall_states_co2["state-name"] == "North Dakota"]

north_dakota_emissions

,state-name,stateId,value,Rank
33,North Dakota,ND,114.310876,34


# CO2 Emissions from COAL

In [9]:
url = "https://api.eia.gov/v2/co2-emissions/co2-emissions-aggregates/data/"
api_key = config.API_KEY


# base url = url + api_key
url_1 = f"{url}?api_key={api_key}"

params_coal = {
    "frequency": "annual",
    "data[0]": "value",
    "facets[sectorId][]": "TT",
    "facets[stateId][]": [
        "AK", "AL", "AR", "AZ", "CA", "CO", "CT", "DE", "FL", "GA", 
        "HI", "IA", "ID", "IL", "IN", "KS", "KY", "LA", "MA", "MD", 
        "ME", "MI", "MN", "MO", "MS", "MT", "NC", "ND", "NE", "NH", 
        "NJ", "NM", "NV", "NY", "OH", "OK", "OR", "PA", "RI", "SC", 
        "SD", "TN", "TX", "UT", "VA", "VT", "WA", "WI", "WV", "WY"
    ],
    "facets[fuelId][]": "CO",
    "start": 2019,
    "end": 2019,
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
}




# Check the response
test_API(url_1)

# filtered response coal
filtered_response_coal_2019 = requests.get(url_1, params=params_coal).json()


# total filtered response
print("\nrecord count for coal response: ", filtered_response_coal_2019['response']['total'])


# extract the actual data
data_coal_2019 = filtered_response_coal_2019['response']['data']


# view sample data
print("\n\nview the first 2 records of coal:")
data_coal_2019[:2]



API key is working, Response: 200

record count for coal response:  50


view the first 2 records of coal:


[{'period': '2019',
  'sectorId': 'TT',
  'sector-name': 'Total carbon dioxide emissions from all sectors',
  'fuelId': 'CO',
  'fuel-name': 'Coal',
  'stateId': 'AK',
  'state-name': 'Alaska',
  'value': '1.685664',
  'value-units': 'million metric tons of CO2'},
 {'period': '2019',
  'sectorId': 'TT',
  'sector-name': 'Total carbon dioxide emissions from all sectors',
  'fuelId': 'CO',
  'fuel-name': 'Coal',
  'stateId': 'WY',
  'state-name': 'Wyoming',
  'value': '39.227048',
  'value-units': 'million metric tons of CO2'}]

### save data_coal_2019 to csv file 

In [10]:
write_data_to_csv(data_coal_2019, 
                  directory='Data/CO2/', 
                  filename='CoalEmissionsByState2019.csv')

Data written to Data/CO2/CoalEmissionsByState2019.csv


## ND coal emission

In [11]:
# coal
df_coal = pd.read_csv('Data/CO2/CoalEmissionsByState2019.csv')
df_coal.head()

,period,sectorId,sector-name,fuelId,fuel-name,stateId,state-name,value,value-units
0,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,AK,Alaska,1.685664,million metric tons of CO2
1,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,WY,Wyoming,39.227048,million metric tons of CO2
2,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,AR,Arkansas,22.931031,million metric tons of CO2
3,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,AZ,Arizona,24.645333,million metric tons of CO2
4,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,CA,California,2.952393,million metric tons of CO2


In [12]:
df_coal.tail()

,period,sectorId,sector-name,fuelId,fuel-name,stateId,state-name,value,value-units
45,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,VT,Vermont,0.000000,million metric tons of CO2
46,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,WA,Washington,7.730214,million metric tons of CO2
47,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,WI,Wisconsin,26.785123,million metric tons of CO2
48,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,WV,West Virginia,59.383945,million metric tons of CO2
49,2019,TT,Total carbon dioxide emissions from all sectors,CO,Coal,AL,Alabama,30.226668,million metric tons of CO2


## Top coal emission states 

In [13]:
#def emission_Rank_Calc(df, value_column='value', group_by_column='state-name', top_n=10):

top_10_states = emission_Rank_Calc(df_coal, top_n=10)

top_10_states

,state-name,stateId,value,Rank
42,Texas,TX,94.930237,1
13,Indiana,IN,78.152084,2
47,West Virginia,WV,59.383945,3
12,Illinois,IL,56.495691,4
34,Ohio,OH,56.452510,5
24,Missouri,MO,55.908627,6
16,Kentucky,KY,54.937069,7
37,Pennsylvania,PA,53.811513,8
21,Michigan,MI,42.690697,9
49,Wyoming,WY,39.227048,10


## North Dakota's Rank for Coal emission

In [14]:
all_states_coal = emission_Rank_Calc(df_coal, top_n=50)

all_states_coal.head()

,state-name,stateId,value,Rank
42,Texas,TX,94.930237,1
13,Indiana,IN,78.152084,2
47,West Virginia,WV,59.383945,3
12,Illinois,IL,56.495691,4
34,Ohio,OH,56.452510,5


In [15]:
ND_coal_rank = all_states_coal[all_states_coal["state-name"] == "North Dakota"]

ND_coal_rank

,state-name,stateId,value,Rank
33,North Dakota,ND,35.571869,11


In [20]:
all_states_coal.head()

,state-name,stateId,value,Rank
42,Texas,TX,94.930237,1
13,Indiana,IN,78.152084,2
47,West Virginia,WV,59.383945,3
12,Illinois,IL,56.495691,4
34,Ohio,OH,56.452510,5


# some visuals 

In [31]:
import plotly.express as px
import pandas as pd
import webbrowser



# Create the bubble plot, use 'abbreviation' to show on the bubbles
fig = px.scatter(all_states_coal, x='Rank', y='value', 
                 size='value', color='value',
                 hover_name='state-name', text='stateId',
                 size_max=60)

# Adjust axis to place highest-ranked bubble in the upper right
fig.update_traces(textposition='top center')
fig.update_layout(xaxis=dict(autorange='reversed'), 
                  title="US States Rank by CO2 emission from Coal (Value = million metric tons)")

# Save the plot as HTML and open it in a browser
fig.write_html("bubble_plot_coal.html")
webbrowser.open("bubble_plot_coal.html")

True

In [29]:
tall_states_co2.head()

,state-name,stateId,value,Rank
42,Texas,TX,1366.830499,1
4,California,CA,716.532710,2
8,Florida,FL,467.235577,3
37,Pennsylvania,PA,437.515984,4
12,Illinois,IL,406.950183,5


In [32]:
import plotly.express as px
import pandas as pd
import webbrowser



# Create the bubble plot, use 'abbreviation' to show on the bubbles
fig = px.scatter(tall_states_co2, x='Rank', y='value', 
                 size='value', color='value',
                 hover_name='state-name', text='stateId',
                 size_max=60)

# Adjust axis to place highest-ranked bubble in the upper right
fig.update_traces(textposition='top center')
fig.update_layout(xaxis=dict(autorange='reversed'), 
                  title="US States Rank by Total CO2 Emission (Value = million metric tons)")

# Save the plot as HTML and open it in a browser
fig.write_html("bubble_plot_totalCo2.html")
webbrowser.open("bubble_plot_totalCo2.html")

True